In [1]:
# Importing modules
try:
    import hsbalance as hs;
except ImportError:
    !pip install hsbalance
    import hsbalance as hs;

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: C:\Users\shoai\anaconda3\envs\myenv\Lib\site-packages\xpress\license\community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


In [2]:
import numpy as np

# Huang test_case

In [3]:
ALPHA_math=[['0.199 @ 284.3', '0.118 @ 298.3'],
       ['0.045 @ 321.6', '0.164 @ 318.2'],
       ['0.095 @ 20.3', '0.054 @ 12.6'],
       ['0.064 @ 162.9', '0.083 @ 28.1']]

A_math=[['42@68'], ['41@221'],['27@153'],['26@325']] 

Convert to complex numbers (cartesian) form

In [4]:
A = hs.convert_matrix_to_cart(A_math)
ALPHA = hs.convert_matrix_to_cart(ALPHA_math)
# A, ALPHA

Adding ALPHA

In [5]:
alpha = hs.Alpha()
alpha.add(direct_matrix=ALPHA)

In [6]:
alpha.check()


Not a square matrix --> no exact solution.

No ill conditioned planes --> ok


In [7]:
print(alpha)


++++++++++++++++++++++++++++++++++++++++
Influence Coefficient Matrix
++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++
Coefficient Values
                Plane 1        Plane 2
Sensor 1  0.199 @ 284.3  0.118 @ 298.3
Sensor 2  0.045 @ 321.6  0.164 @ 318.2
Sensor 3   0.095 @ 20.3   0.054 @ 12.6
Sensor 4  0.064 @ 162.9   0.083 @ 28.1
End of Coefficient Values
++++++++++++++++++++++++++++++++++++++++

                   


## Solving with Least squares:

In [8]:
 # Instantiate least square model

In [9]:
model_LeastSquares = hs.LeastSquares(A, alpha)

In [10]:
W_LeastSquares = model_LeastSquares.solve() #solve

In [11]:
hs.convert_matrix_to_math(W_LeastSquares)

array([['319.223 @ 314.6'],
       ['227.495 @ 103.7']], dtype='<U15')

In [12]:
model_LeastSquares.rmse()

11.5318

## Splitting plane BZ-A

In [13]:
split_BZA = model_LeastSquares.create_split()

In [14]:
angles = np.arange(0,360,45)

In [15]:
split_BZA.split_setup(0, max_number_weights_per_hole=5, holes_available=[angles]
                         ,weights_available=[5,10,20], max_weight_per_plane=319.223)

In [16]:
%%timeit
split_BZA.split_solve()

41.2 ms ± 6.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
split_BZA.split_solve()

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 5., 5.]])

In [18]:
split_BZA.results()

,0,270,315
weights_available,,,
5,0.0,0.0,0.0
10,0.0,0.0,0.0
20,5.0,5.0,5.0


In [19]:
split_BZA.error()

array([77.82988302])

In [20]:
split_BZA.error(options='equ')

array('241.421 @ 315.0', dtype='<U15')

In [21]:
split_BZA.error(options='problem_error') # getting the problem accuracy

array([0.])

In [22]:
split_BZA.update()

C:\Users\shoai\anaconda3\envs\myenv\lib\site-packages\hsbalance\model.py:255: UserWarning: This will change your model optimum solution.Choose confirm=True
  warnings.warn('This will change your model optimum solution.'


array([[224.02137493-227.41570696j],
       [-53.73888265+221.05673966j]])

In [23]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['319.223 @ 314.6'],
       ['227.495 @ 103.7']], dtype='<U15')

In [24]:
split_BZA.update(confirm=True)

array([[170.71067812-170.71067812j],
       [-53.73888265+221.05673966j]])

In [25]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['241.421 @ 315.0'],
       ['227.495 @ 103.7']], dtype='<U15')

In [26]:
model_LeastSquares.rmse()

15.3235

In [27]:
print(model_LeastSquares.info())


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL TYPE
LeastSquares
End of MODEL TYPE
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
INFLUENCE COEFFICIENT MATRIX

++++++++++++++++++++++++++++++++++++++++
Influence Coefficient Matrix
++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++
Coefficient Values
                Plane 1        Plane 2
Sensor 1  0.199 @ 284.3  0.118 @ 298.3
Sensor 2  0.045 @ 321.6  0.164 @ 318.2
Sensor 3   0.095 @ 20.3   0.054 @ 12.6
Sensor 4  0.064 @ 162.9   0.083 @ 28.1
End of Coefficient Values
++++++++++++++++++++++++++++++++++++++++

                   
End of INFLUENCE COEFFICIENT MATRIX
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
+++++++++++++++++++++

## Splitting plane BZ-C

In [28]:
split_BZC = model_LeastSquares.create_split()

In [29]:
angles = np.arange(22.5, 360, 45)

In [30]:
split_BZC.split_setup(1, max_number_weights_per_hole=3, holes_available=[angles]
                         ,weights_available=[10,20], max_weight_per_plane=227)

In [31]:
%%timeit
split_BZC.split_solve()

28.5 ms ± 366 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
split_BZC.split_solve()

array([[0., 4., 0., 0., 0., 0., 0., 0.],
       [0., 0., 9., 0., 0., 0., 0., 0.]])

In [33]:
split_BZC.results()

,67.5,112.5
weights_available,,
10,4.0,0.0
20,0.0,9.0


In [34]:
split_BZC.error()

array([17.80399053])

In [35]:
split_BZC.error(options='equ')

array('210.196 @ 104.8', dtype='<U15')

In [36]:
split_BZC.error(options='problem_error') # getting the problem accuracy

array([0.])

In [37]:
split_BZC.update()

C:\Users\shoai\anaconda3\envs\myenv\lib\site-packages\hsbalance\model.py:255: UserWarning: This will change your model optimum solution.Choose confirm=True
  warnings.warn('This will change your model optimum solution.'


array([[170.71067812-170.71067812j],
       [-53.73888265+221.05673966j]])

In [38]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['241.421 @ 315.0'],
       ['227.495 @ 103.7']], dtype='<U15')

In [39]:
split_BZC.update(confirm=True)

array([[170.71067812-170.71067812j],
       [-53.57568053+203.25349715j]])

In [40]:
hs.convert_matrix_to_math(W_LeastSquares)  # The new W_LeastSquares

array([['241.421 @ 315.0'],
       ['210.196 @ 104.8']], dtype='<U15')

In [41]:
model_LeastSquares.rmse()

14.9993

In [42]:
print(model_LeastSquares.info())


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
MODEL TYPE
LeastSquares
End of MODEL TYPE
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
INFLUENCE COEFFICIENT MATRIX

++++++++++++++++++++++++++++++++++++++++
Influence Coefficient Matrix
++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++
Coefficient Values
                Plane 1        Plane 2
Sensor 1  0.199 @ 284.3  0.118 @ 298.3
Sensor 2  0.045 @ 321.6  0.164 @ 318.2
Sensor 3   0.095 @ 20.3   0.054 @ 12.6
Sensor 4  0.064 @ 162.9   0.083 @ 28.1
End of Coefficient Values
++++++++++++++++++++++++++++++++++++++++

                   
End of INFLUENCE COEFFICIENT MATRIX
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

                   
+++++++++++++++++++++

In [43]:
model_LeastSquares.split_instance   # We can see the splitting instances that override the optimum solution